In [2]:
# read raw eeg data and ouput targeted coherence epoches

from sankar_read_vmrk_events import sankar_read_vmrk_events
import mne
import os
import numpy as np
from mne.io.brainvision import brainvision
import warnings
warnings.filterwarnings('ignore')
import scipy.io
import listen_italian_functions

data_path = os.path.dirname(os.path.dirname(os.getcwd()))


s = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
     'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
     'Silvia','Silvia2','Tommaso']

Tmin = 0
Tmax = 3.51
trial_len = 2

for subject_name in s:
    save_path = data_path + '/analysis/python/data/'+subject_name
    raw_fname = data_path + '/exp/data/eeg_original/'+subject_name+'.vhdr'

    raw = mne.io.read_raw_brainvision(raw_fname,preload=True)
    #raw.set_eeg_reference('average', projection=True)  # set EEG average reference
    raw.pick_types(meg=False, eeg=True, eog=True, stim=True)

    raw.set_channel_types(mapping={'vEOGup': 'eog'})
    raw.set_channel_types(mapping={'vEOGdown': 'eog'})
    raw.set_channel_types(mapping={'LHEOG': 'eog'})
    raw.set_channel_types(mapping={'RHEOG': 'eog'})
    raw.rename_channels(mapping={'vEOGup': 'vEOGup'})
    raw.rename_channels(mapping={'vEOGdown': 'vEOGdown'})
    raw.rename_channels(mapping={'LHEOG': 'LHEOG'})
    raw.rename_channels(mapping={'RHEOG': 'RHEOG'})

    #events
    raw_fname = data_path + '/exp/data/eeg_original/'+subject_name+'.vmrk'
    events = sankar_read_vmrk_events(raw_fname)


    # for ferrara brainamp acticap
    montage = mne.channels.read_montage('easycap-M1')
    ch_names = montage.ch_names
    remove_ch = ["Fpz", "Iz","F9", "F10", "P9", "P10", "O9", "O10", "AFz","FT9","FT10","TP9","TP10","PO9","PO10"]
    a = np.nonzero(np.in1d(ch_names, remove_ch))[0]
    ch_names = np.delete(ch_names,a)
    montage = mne.channels.read_montage('easycap-M1',ch_names)
    a=raw.set_montage(montage)

    # cut 500 + speech onset - speech offset + 500ms from the original recordings
    a=np.where(events[:,2]==5)[0]
    b=np.where(events[:,2]==6)[0]
    a=np.sort(np.append(a,b))
    selected_portion = events[a]

    X=500 #500 ms
    i=0
    a1 =  np.array([(selected_portion[:,0][i]-X),0,100,0])
    a2 = np.array([(selected_portion[:,0][i]),0,105,1])
    a3 = np.array([(selected_portion[:,0][i+1]),0,106,0])
    a4 = np.array([(selected_portion[:,0][i+1]+X),0,200,0])
    A = np.vstack((a1,a2,a3,a4))

    k=2

    for i in range(2, len(selected_portion)-1,2):
        a1 =  np.array([(selected_portion[:,0][i]-X),0,100,0])
        a2 = np.array([(selected_portion[:,0][i]),0,105,k])
        a3 = np.array([(selected_portion[:,0][i+1]),0,106,0])
        a4 = np.array([(selected_portion[:,0][i+1]+X),0,200,0])
        A = np.vstack((A,a1,a2,a3,a4))
        k=k+1

    selected_portion = A[:,0:3]

    # add this events to raw and delete the original one
    stim_data = np.zeros((1, len(raw.times)))
    info = mne.create_info(['Trigger'], raw.info['sfreq'], ['stim'])
    stim_raw = mne.io.RawArray(stim_data, info)

    raw.add_channels([stim_raw], force_update_info=True)
    raw.add_events(selected_portion, stim_channel='Trigger')
    raw.drop_channels(['STI 014'])

    info = mne.create_info(['trial_no'], raw.info['sfreq'], ['misc'])
    stim_raw = mne.io.RawArray(stim_data, info)
    a=A[:,[0, 1, 3]]
    raw.add_channels([stim_raw], force_update_info=True)
    raw.add_events(a, stim_channel='trial_no')

    a=mne.find_events(raw)
    print(a[:10])


    #cut portion of from the original recordings
    X = 0.05
    raw1 = raw.copy().crop(selected_portion[:,0][0]/1000 - X, selected_portion[:,0][3]/1000 + X)
    for i in range(4, len(selected_portion),4):
        raw2 = raw.copy().crop(selected_portion[:,0][i]/1000 - X, selected_portion[:,0][i+3]/1000 + X)
        raw1.append([raw2])
        print(i)

    event_id = {'Speech_onset': 105,'Speech_offset': 106}

    # check and remove the edge boundary annotations and make all of it continues recordings
    a=mne.find_events(raw1)
    print(a[:10])
    raw=raw1
    raw.annotations.delete(np.arange(0,raw.annotations.description.shape[0]))

    #   Set up pick list: EEG + STI 014 - bad channels (modify to your needs)
    include = ['Trigger']  # or stim channels ['STI 014']
    raw.info['bads'] = ['RM']  
    # pick eeg channels
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=True, eog=False, misc=True,exclude='bads')
    raw.save(save_path +'_raw.fif',picks=picks,overwrite=True)
    
    
    # extract trials of tmax second and remove the wrong answer trials and seperate them in three conditions   
    raw_fname = data_path + '/analysis/python/data/'+subject_name+'_raw.fif'
    raw = mne.io.read_raw_fif(raw_fname,preload=True)
    raw_fname = data_path + '/analysis/behaviour/data/subject/'+subject_name+'_behaviour.mat'
    mat = scipy.io.loadmat(raw_fname)
    epochs = listen_italian_functions.epoch(raw, mat,Tmin, Tmax)
    save_path = data_path + '/analysis/python/data/'+subject_name+'-coh-epo-'+str(Tmin)+'-'+str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs.save(save_path)

Extracting parameters from D:\projects\current\listen_italian_motor_entrainment/exp/data/eeg_original/Alice.vhdr...
Setting channel info structure...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
Reading 0 ... 2015319  =      0.000 ...  2015.319 secs...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=2015320
    Range : 0 ... 2015319 =      0.000 ...  2015.319 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2015320
    Range : 0 ... 2015319 =      0.000 ...  2015.319 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[26117     0   100]
 [26617   

Setting channel info structure...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
Reading 0 ... 2003259  =      0.000 ...  2003.259 secs...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=2003260
    Range : 0 ... 2003259 =      0.000 ...  2003.259 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2003260
    Range : 0 ... 2003259 =      0.000 ...  2003.259 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[41400     0   100]
 [41900     0   105]
 [51051     0   106]
 [51551     0   200]
 [55788     0   100]
 [56288     0   105]
 [59781     0   106]


Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
Reading 0 ... 2018919  =      0.000 ...  2018.919 secs...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=2018920
    Range : 0 ... 2018919 =      0.000 ...  2018.919 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2018920
    Range : 0 ... 2018919 =      0.000 ...  2018.919 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[21200     0   100]
 [21700     0   105]
 [27110     0   106]
 [27610     0   200]
 [31536     0   100]
 [32036     0   105]
 [35590     0   106]
 [36090     0   200]
 [40274     0

Reading 0 ... 1988799  =      0.000 ...  1988.799 secs...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=1988800
    Range : 0 ... 1988799 =      0.000 ...  1988.799 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1988800
    Range : 0 ... 1988799 =      0.000 ...  1988.799 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[21715     0   100]
 [22215     0   105]
 [25604     0   106]
 [26104     0   200]
 [30302     0   100]
 [30802     0   105]
 [35684     0   106]
 [36184     0   200]
 [40629     0   100]
 [41129     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156

Reading 0 ... 2015359  =      0.000 ...  2015.359 secs...
Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=2015360
    Range : 0 ... 2015359 =      0.000 ...  2015.359 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2015360
    Range : 0 ... 2015359 =      0.000 ...  2015.359 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[18550     0   100]
 [19050     0   105]
 [27528     0   106]
 [28028     0   200]
 [32640     0   100]
 [33140     0   105]
 [35508     0   106]
 [36008     0   200]
 [39889     0   100]
 [40389     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156

...

Currently, 1 trigger(s) will be dropped, such as [actiCAP not connected]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['RM']. Their position has been left untouched.
Creating RawArray with float64 data, n_channels=1, n_times=1910700
    Range : 0 ... 1910699 =      0.000 ...  1910.699 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1910700
    Range : 0 ... 1910699 =      0.000 ...  1910.699 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[24992     0   100]
 [25492     0   105]
 [32925     0   106]
 [33425     0   200]
 [37203     0   100]
 [37703     0   105]
 [39863     0   106]
 [40363     0   200]
 [44133     0   100]
 [44633     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
2

Creating RawArray with float64 data, n_channels=1, n_times=1977260
    Range : 0 ... 1977259 =      0.000 ...  1977.259 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1977260
    Range : 0 ... 1977259 =      0.000 ...  1977.259 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[17762     0   100]
 [18262     0   105]
 [24973     0   106]
 [25473     0   200]
 [29645     0   100]
 [30145     0   105]
 [34753     0   106]
 [35253     0   200]
 [39149     0   100]
 [39649     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
5

Creating RawArray with float64 data, n_channels=1, n_times=2227360
    Range : 0 ... 2227359 =      0.000 ...  2227.359 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2227360
    Range : 0 ... 2227359 =      0.000 ...  2227.359 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[24435     0   100]
 [24935     0   105]
 [27979     0   106]
 [28479     0   200]
 [33348     0   100]
 [33848     0   105]
 [38422     0   106]
 [38922     0   200]
 [43743     0   100]
 [44243     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
5

    Range : 0 ... 2019479 =      0.000 ...  2019.479 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=2019480
    Range : 0 ... 2019479 =      0.000 ...  2019.479 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[39628     0   100]
 [40128     0   105]
 [45532     0   106]
 [46032     0   200]
 [50202     0   100]
 [50702     0   105]
 [54242     0   106]
 [54742     0   200]
 [59533     0   100]
 [60033     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572


    Range : 0 ... 1981279 =      0.000 ...  1981.279 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=1981280
    Range : 0 ... 1981279 =      0.000 ...  1981.279 secs
Ready.
800 events found
Event IDs: [100 105 106 200]
[[21483     0   100]
 [21983     0   105]
 [29407     0   106]
 [29907     0   200]
 [34474     0   100]
 [34974     0   105]
 [37117     0   106]
 [37617     0   200]
 [41488     0   100]
 [41988     0   105]]
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348
352
356
360
364
368
372
376
380
384
388
392
396
400
404
408
412
416
420
424
428
432
436
440
444
448
452
456
460
464
468
472
476
480
484
488
492
496
500
504
508
512
516
520
524
528
532
536
540
544
548
552
556
560
564
568
572
